In [110]:
%load_ext autoreload
%autoreload 2

import sys, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
# results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
results_df_old = pd.DataFrame(columns = ['run_id'])
results_df_old

,run_id


In [112]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

run_dirs =  np.array([[run_dir.split('-')[-1], run_dir] for run_dir in os.listdir(wandb_dir) if run_dir.startswith('run')])
if not len(run_dirs) == 0:
    assert len(run_dirs[:, 0]) == len(set(run_dirs[:, 0]))
    run_dirs = {run_id: run_dir for run_id, run_dir in run_dirs}
    run_dirs_df = pd.DataFrame(run_dirs.items(), columns = ['run_id', 'run_dir'])
else:
    run_dirs_df = pd.DataFrame(columns = ['run_id', 'run_dir'])
run_dirs_df

,run_id,run_dir


In [113]:
exclude_ids = ['1jjq8ar4']

In [116]:
tags = 'dev'

api = wandb.Api()
runs = api.runs('eliasdubbeldam/symlie', filters={'tags': tags, 'state': 'finished'})
for i, run in enumerate(runs):
    print(run.name, run.id)
    # if run.id in exclude_ids: run.delete()

In [120]:
config_list = []

pbar = tqdm(runs)
for run in pbar:
    pbar.set_description(f'Retreiving wandb {str(id)}')
    
    id = run.id

    # Exceptions
    if run.state != 'finished':
        continue
    if id in results_df_old['run_id'].values:
        continue
    if id in exclude_ids:
        continue


    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Retreive metrics from wandb run
    history = run.history()
    metrics = ['train_loss', 'val_loss', 'test_loss']
    for metric in metrics:
        config[metric] = np.array(history[metric].dropna())

    # Retreive arrays from wandb files
    npu = NumpyUtils(dir = os.path.join(wandb_dir, run_dirs[id], 'files'))
    run_arrays = npu.load_all()
    for filename, array in run_arrays.items():
        config[filename] = array
    

    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

0it [00:00, ?it/s]


""


In [121]:
results_df_new = pd.concat([results_df_old, results_df])
results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))